In [ ]:
%pip install numpy pandas nibabel nilearn networkx python-louvain tqdm scipy

import numpy as np
from scipy.signal.windows import hann, hamming, blackman
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

TS_DIR = Path("/Users/hetjivani/NIAR/Neuro-Imaging-Analysis-and-Research/GroupE/Extraction/processed/timeseries")
WIN_DIR = Path("processed/windows")
WIN_DIR.mkdir(parents=True, exist_ok=True)

cfgs = [
    # Different window lengths
    dict(name="w30_s15_rect", length=30, step=15, win=None),
    dict(name="w30_s15_hann", length=30, step=15, win="hann"),
    dict(name="w45_s15_rect", length=45, step=15, win=None),
    dict(name="w45_s15_hann", length=45, step=15, win="hann"),
    dict(name="w60_s15_rect", length=60, step=15, win=None),
    dict(name="w60_s15_hann", length=60, step=15, win="hann"),
    dict(name="w60_s30_rect", length=60, step=30, win=None),
    dict(name="w60_s30_hann", length=60, step=30, win="hann"),
    dict(name="w90_s30_rect", length=90, step=30, win=None),
    dict(name="w90_s30_hann", length=90, step=30, win="hann"),
    dict(name="w90_s45_rect", length=90, step=45, win=None),
    dict(name="w90_s45_hann", length=90, step=45, win="hann"),
    dict(name="w120_s30_rect", length=120, step=30, win=None),
    dict(name="w120_s30_hann", length=120, step=30, win="hann"),
    dict(name="w120_s60_rect", length=120, step=60, win=None),
    dict(name="w120_s60_hann", length=120, step=60, win="hann"),

    # Additional window types for best candidates
    dict(name="w60_s30_hamming", length=60, step=30, win="hamming"),
    dict(name="w90_s30_hamming", length=90, step=30, win="hamming"),
    dict(name="w60_s30_blackman", length=60, step=30, win="blackman"),
    dict(name="w90_s30_blackman", length=90, step=30, win="blackman"),
]


def make_windows(ts, length, step, wtype):
    T, R = ts.shape
    n = 1 + (T - length) // step
    out = np.empty((n, length, R), dtype=np.float32)

    if wtype == "hann":
        w = hann(length, sym=False).astype(np.float32)[:, None]
    elif wtype == "hamming":
        w = hamming(length, sym=False).astype(np.float32)[:, None]
    elif wtype == "blackman":
        w = blackman(length, sym=False).astype(np.float32)[:, None]

    for i in range(n):
        s = i * step
        seg = ts[s: s + length].copy()
        if wtype is not None:
            seg *= w
        out[i] = seg
    return out


def cov_dist(a, b):
    return np.linalg.norm(a - b, 'fro') / np.linalg.norm(a, 'fro')


def conn_metric(windows):
    # Compute correlation matrices for each window
    corrs = []
    for w in windows:
        c = np.corrcoef(w.T)
        corrs.append(c)

    # Calculate variability across windows (dynamic range)
    corr_std = np.std(corrs, axis=0)
    return np.mean(corr_std)


rows = []

for ts_path in tqdm(sorted(list(TS_DIR.glob("*.npy")))):
    ts = np.load(ts_path)  # (T, 200)
    full_cov = np.cov(ts.T, dtype=np.float64)

    for cfg in cfgs:
        wins = make_windows(ts, cfg["length"], cfg["step"], cfg["win"])

        # Covariance preservation metric
        covs = np.mean([np.cov(w.T, dtype=np.float64) for w in wins], axis=0)
        cov_loss = cov_dist(full_cov, covs)

        # Dynamic connectivity metric
        dyn_metric = conn_metric(wins)

        rows.append({
            "file": ts_path.stem,
            "cfg": cfg["name"],
            "windows": wins.shape[0],
            "cov_loss": cov_loss,
            "dyn_metric": dyn_metric,
        })

        # Save full dataset for all configurations
        np.save(
            WIN_DIR / f'{ts_path.stem}_{cfg["name"]}.npy',
            wins,
            allow_pickle=False
        )

df = pd.DataFrame(rows)

if not df.empty:
    # Calculate combined score (lower cov_loss and higher dyn_metric is better)
    df["score"] = (1 - df["cov_loss"]) + df["dyn_metric"]

    # Summary of performance
    summary = df.groupby("cfg").agg({
        "windows": "mean",
        "cov_loss": ["mean", "std"],
        "dyn_metric": ["mean", "std"],
        "score": "mean"
    }).round(4)

    summary = summary.sort_values(("score", "mean"), ascending=False)
    print(summary)
    print("\nTop 5 window configurations:")
    print(summary.head(5))
else:
    print("No data available. Please check if there are .npy files in TS_DIR.")


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


100%|██████████| 84/84 [00:03<00:00, 22.40it/s]

                 windows cov_loss         dyn_metric           score
                    mean     mean     std       mean     std    mean
cfg                                                                 
w30_s15_rect        14.0   0.3142  0.0794     0.4324  0.0204  1.1182
w45_s15_rect        13.0   0.2338  0.0633     0.3402  0.0210  1.1064
w60_s15_rect        12.0   0.2311  0.0501     0.2768  0.0205  1.0457
w60_s30_rect         6.0   0.2563  0.0573     0.2726  0.0205  1.0163
w90_s45_rect         4.0   0.2113  0.0340     0.1938  0.0187  0.9825
w90_s30_rect         5.0   0.3033  0.0531     0.1857  0.0177  0.8824
w120_s30_rect        4.0   0.3351  0.0636     0.1223  0.0161  0.7872
w30_s15_hann        14.0   0.7288  0.0244     0.5093  0.0151  0.7805
w45_s15_hann        13.0   0.6845  0.0292     0.4337  0.0165  0.7493
w60_s15_hann        12.0   0.6577  0.0354     0.3759  0.0172  0.7183
w60_s30_hamming      6.0   0.6420  0.0424     0.3564  0.0181  0.7144
w60_s30_hann         6.0   0.6648 

In [10]:
df.to_csv(WIN_DIR / "window_benchmark.csv", index=False)
print(f"\nResults saved to {WIN_DIR / 'window_benchmark.csv'}")
print(df.shape)


Results saved to processed/windows/window_benchmark.csv
(1680, 6)
